In [1]:
from elasticsearch import Elasticsearch
import pandas as pd

In [2]:
#Navigate to Elasticsearch connect to local host
es = Elasticsearch("http://localhost:9200")
es.info().body

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fdf2276e2c0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [3]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
print(es.ping())

TypeError: NodeConfig.__init__() missing 1 required positional argument: 'scheme'

In [5]:
es = Elasticsearch([{'host': 'localhost', 'port': '9200', "scheme": "https"}])
print(es.ping())

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
#Read data file and place into pandas dataframe
df = (
    pd.read_csv("/home/namorak/scripps-research-test/Data/sample_interaction_data.tsv",sep="\t",lineterminator='\n')
    .dropna()
    # .sample(5000, random_state=42)
    .reset_index()
)
#pd.set_option('display.max_colwidth', None)
print(df.head())

In [ ]:
#Create index and mappings
mappings = {
    "properties": {
        "index": {"type": "long "},
        "gene_name": {"type": "text"}, 
        "gene_claim_name": {"type": "text"},
        "entrez_id": {"type": "long "},
        "interaction_claim_source": {"type": "text"},
        "interaction_types": {"type": "text"},
        "drug_claim_primary_name": {"type": "text"},    
        "drug_name": {"type": "text"},      
        "drug_concept_id": {"type": "text"},
        "interaction_group_score": {"type": "long "},    
        "PMIDs": {"type": "long "}
    }
}

#es.indices.create(index="medications", mappings=mappings)

In [ ]:
#Add data to index
for i, row in df.iterrows():
    doc = {
        "index": row["index"],
        "gene_name": row["gene_name"], 
        "gene_claim_name": row["gene_claim_name"],
        "entrez_id": row["entrez_id"],
        "interaction_claim_source": row["interaction_claim_source"],
        "interaction_types": row["interaction_types"],
        "drug_claim_primary_name": row["drug_claim_primary_name"],    
        "drug_name": row["drug_name"],      
        "drug_concept_id": row["drug_concept_id"],
        "interaction_group_score": row["interaction_group_score"],    
        "PMIDs": row["PMIDs"]
    }
            
es.index(index="medications", id=i, document=doc)

In [ ]:
#Store bulk data in list
bulk_data = []
for i,row in df.iterrows():
    bulk_data.append(
        {
            "_index": "medications",
            "_id": i,
            "_source": {        
                "index": row["index"],
                "gene_name": row["gene_name"], 
                "gene_claim_name": row["gene_claim_name"],
                "entrez_id": row["entrez_id"],
                "interaction_claim_source": row["interaction_claim_source"],
                "interaction_types": row["interaction_types"],
                "drug_claim_primary_name": row["drug_claim_primary_name"],    
                "drug_name": row["drug_name"],      
                "drug_concept_id": row["drug_concept_id"],
                "interaction_group_score": row["interaction_group_score"],    
                "PMIDs": row["PMIDs"],
            }
        }
    )
bulk(es, bulk_data)


es.indices.refresh(index="medications")
es.cat.count(index="medications", format="json")

In [ ]:
resp = es.search(
    index="medications",
    query={
            "bool": {
                "must": {
                    "match_phrase": {
                        "gene_name": "CDK7",
                    }
                },
                "filter": {"bool": {"must_not": {"match_phrase": {"column name": "data field"}}}},
            },
        },            
)
resp.body